![Cyberpunk Agent](main.webp "")

### Introduction: Why AI Agents Matter
<br>
Before diving into the coding process, it's essential to understand the purpose of the program we aim to build. We need to identify the problem at hand and the business value we can derive from solving it...

Lol, who am I kidding, we're not at work here, so let's focus on picking problems that are ***fun*** to solve and let the marketing people worry about value creation.

Jokes aside, today our goal is to learn how to overcome the traditional limitations of Large Language Models (LLMs) by building Agents that are able to use *custom tools*.  

While LLMs like GPT-4 are incredibly powerful and capable of performing various tasks such as answering questions, summarizing text, and reasoning about complex topics, they have **significant limitations** when used in isolation. What are those limitations, one may ask?


1. **Lack of Real-Time Data**  
   - LLMs are trained on static datasets and *cannot fetch live information* (e.g., stock prices, weather updates, or breaking news).  
   - This leads to outdated or incomplete responses.  
<br>
2. **Inability to Perform Actions**  
   - An LLM can tell you how to book a flight but *can’t actually book it*—it lacks the ability to interact with external systems.  
   - Agents, on the other hand, can call APIs, interact with databases, and automate tasks.  
<br>
3. **Limited Memory & Context**  
   - While LLMs have a context window, they *forget past interactions beyond a limit* and struggle with long-term tasks.  
   - Agents can store and retrieve relevant information, improving coherence over long workflows.  
<br>
4. **Mathematical & Computational Weaknesses**  
   - LLMs can make **math mistakes** or fail in complex computations.  
   - Agents can call external computation tools (e.g., a Python script or a calculator API) for precise results.  



#### **How Agents Solve These Problems**  
*AI agents* address many of the limitations of traditional language models by integrating external tools, allowing for greater accuracy, real-time data retrieval, and interaction with external systems. In this tutorial, we will explore how to create and integrate custom tools with LLMs to build more robust and functional AI agents. 

Specifically, we will develop:

- 🧮 Mathematical computation tools capable of producing precise results, overcoming the tendency of LLMs to generate incorrect numerical answers (with a clear example of ChatGPT's limitation that you can test against your Agent's tool) 
- 📈 Real-time stock market query tools to retrieve up-to-date financial information. 
- 🔍 Web-search tools to access the latest research and trends, ensuring information is current. 

All implementations will rely entirely on open-source libraries (big love to Hugging Face ❤️) and will run locally on a standard machine with no more than 12GB of VRAM. While many existing tutorials focus on cloud-based implementations via [HuggingFace's Serverless API](https://huggingface.co/docs/api-inference/en/index), I want to demonstrate how to get things running without relying on the cloud.



## Building Open Source Agents using HuggingFace's SmolAgents


To monitor GPU utilization in real-time, especially when running resource-intensive machine learning tasks, you can leverage the `nvidia-smi` command. On Ubuntu, simply uncomment and execute the following command in your terminal (provided that you have an Nvidia GPU):

```bash
watch -n 1 nvidia-smi
```

This command offers a live, updating snapshot of your GPU's performance, enabling you to monitor key metrics such as memory utilization and temperature. The `watch -n 1` component ensures that this information is refreshed every second. This is particularly useful for debugging, optimizing your machine learning workflows, and identifying specific commands or models that lead to critical "Out Of Memory" errors.

### Imports & Setup

In this initial code block, we start by importing the necessary libraries to construct our local AI agent. Let's break down each import:

- `typing`: We import `Any` from the `typing` module to provide type hints for our function signatures, which is a critical part of creating useful tools that our agents that use.

- `smolagents`: From the `smolagents` library, we import:
    - `Tool`: A base class for defining custom tools that our agent can use.
    - `CodeAgent`: The core class for creating agents capable of generating and executing code.
    - `TransformersModel`: A class to load and manage transformer models.
    - `tool`: A decorator to easily register functions as tools for the agent.
    - `DuckDuckGoSearchTool`: A pre-built tool that allows the agent to perform web searches using DuckDuckGo.
- `transformers`: We import `AutoTokenizer` from the `transformers` library, which will be used to process text data for our models.
- `yaml`: This library is imported to handle YAML configuration files, allowing us to define agent settings in a structured format.
- `yfinance`: We import `yfinance` to fetch stock market data.
- `datetime` and `pytz`: These are used for handling date and time information, which will be useful when working with financial data.

These imports set the stage for building a powerful AI agent that can interact with its environment through code execution and web searches.

In [12]:
#| echo: false
#| output: asis
with open('requirements.txt') as f:
    requirements = f.read().strip()
    
print(f'You can see which dependencies are used for running this notebook by either simply hovering over <span style="text-decoration: underline dotted; cursor: help; color:rgb(255, 105, 230);" title="{requirements}">requirements.txt</span> or by visiting <a href="https://github.com/Jonathanmoatti/Jonathanmoatti.github.io/tree/main/posts/local_agents">this blog post\'s GitHub page</a> and accessing the file.')

You can see which dependencies are used for running this notebook by either simply hovering over <span style="text-decoration: underline dotted; cursor: help; color:rgb(255, 105, 230);" title="markdownify
smolagents
requests
duckduckgo_search
pandas
yfinance">requirements.txt</span> or by visiting <a href="https://github.com/Jonathanmoatti/Jonathanmoatti.github.io/tree/main/posts/local_agents">this blog post's GitHub page</a> and accessing the file.


In [ ]:
from typing import Any
from smolagents.tools import Tool
from smolagents import CodeAgent, TransformersModel, tool, DuckDuckGoSearchTool
from transformers import AutoTokenizer
import yaml

import yfinance as yf
from datetime import datetime
import pytz


### Define your prompt template
Load up the adequate yaml prompt template. This one is compatible with the smolagents library and the model we will use. It was taken directly from [HuggingFace's Agents Course](https://huggingface.co/agents-course)

In [ ]:
# Setting up the predefined prompts
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

### **Understanding AI Tool Creation with SmolAgents**

When building AI agents, **defining tools properly is crucial** for ensuring smooth integration with frameworks like [SmolAgents](https://huggingface.co/docs/smolagents/en/index).  
The first tool we will build today uses the `yfinance` library to check the status (price) of a given ticker on the North American stock market. 

#### **Creating your first tool**
The function **`check_NA_market_status`** is decorated with `@tool`, marking it as a callable tool within the agent framework. This tool checks whether the stock market is open and returns the latest stock price if open, or the last closing price if closed.

1. **Appropriate type-hinting is Essential**  
   - The function takes a **single string argument (`ticker: str`)** and returns a **string (`-> str`)**.  
   - This typing ensures that the agent understands the expected input and output format.  
   - Without this, the function may not work correctly in an AI pipeline.

2. **The Importance of a Docstring**  
   - The function is documented with a **clear, structured docstring** explaining:  
     - **What it does** (checks if the market is open and returns the appropriate stock price).  
     - **What arguments it takes** (`ticker` – the stock ticker symbol).  
     - **What it returns** (a formatted string with market status and price).  
   - AI agents rely on this information to understand the function’s role. When/if you build your own tool, it is critical to keep the same docstring template for your code to be compatible with the smolagents library.



##### **Key Takeaways for developping tools**
 

- **Follow the function template:** Use clear type hints and structured docstrings.  
- **Ensure proper decoration:** The `@tool` decorator registers the function within the SmolAgents framework.  
- **Keep return types simple:** AI models parse these results, so returning formatted strings helps with interpretability.  

This structured approach makes it easier for Agents to *call tools correctly, understand their functions, and use them effectively in reasoning and decision-making.*

In [ ]:
from smolagents import tool
# first tool
@tool
def check_NA_market_status(ticker: str) -> str:
    """
    A tool that checks if the stock market is open for the given ticker and returns the current price if open,
    or the previous close price if the market is closed.
    Args:
        ticker: The stock ticker symbol (e.g., 'AAPL', 'GOOG').
    Returns:
        A message indicating whether the market is open or closed, 
        and the price of the stock at the respective time.
    """

    # Get the current time in Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    current_time = now.time()
    current_day = now.weekday()  # Monday is 0 and Sunday is 6

    market_open_time = datetime.strptime("09:30:00", "%H:%M:%S").time()
    market_close_time = datetime.strptime("16:00:00", "%H:%M:%S").time()

    if current_day < 5 and market_open_time <= current_time <= market_close_time:
        stock = yf.Ticker(ticker)
        # Request intraday data: 1-day period with 1-minute interval
        current_price = stock.history(period="1d", interval="1m")['Close'].iloc[-1]
        return f"Market is open. Current price of {ticker}: ${current_price:.2f}"
    else:
        stock = yf.Ticker(ticker)
        previous_close = stock.history(period="1d")['Close'].iloc[-1]
        return f"Market is closed. Previous close price of {ticker}: ${previous_close:.2f}"





### **Bypassing LLM's mathematical computation limitations**

The `fibonacci_tool` function is a custom tool designed to compute the nth Fibonacci number. It is decorated with @tool, which allows it to be seamlessly integrated into an AI agent using the SmolAgents framework.

LLMs, while powerful, often struggle with mathematical computations, especially recursive sequences like Fibonacci numbers. Instead of relying on the LLM's built-in reasoning (which often returns erroneous results even with the most powerful models like GPT-4o and o1 *try it for yourself*), this tool:

1. Provides a deterministic and accurate calculation of Fibonacci numbers.
2. Is significantly more efficient than recursive implementations, using an iterative approach to avoid excessive function calls.
3. Ensures the AI agent always returns correct numerical results rather than relying on approximate or hallucinated values.
<br>

In [5]:
# second tool
@tool
def fibonacci_tool(n: int) -> int:
    """
    A simple tool that returns the nth Fibonacci number.

    Args:
        n: The index of the Fibonacci sequence to retrieve (int)

    Returns:
        int: The nth Fibonacci number.
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

### **Bypassing the LLM's training date cutoff**

Finally, we create a tool that leverages `DuckDuckGoSearch` to fetch up-to-date information from the web. 

This tool empowers our agent with the ability to quickly retrieve relevant information on demand. Since models training have cutoff dates, this tool will allow us to bypass this cutoff limitation by researching the web for up to date answers to any query we may have.

In [6]:
# third tool
@tool
def duckduckgo_search(query: str) -> str:
    """
    A tool that performs a real-time DuckDuckGo search for the given query and returns the top results.
    
    Args:
        query: The search query string (e.g., "Latest AI research 2025").
        
    Returns:
        A string containing the search results. If the search tool returns a formatted string,
        it is returned directly; otherwise, if it returns a list of dictionaries with 'title' and 'url'
        keys, the results are formatted for readability.
    """
    search_tool = DuckDuckGoSearchTool()
    results = search_tool(query)
    
    if isinstance(results, str):
        return results
    
    formatted_results = "\n".join(
        [f"{item.get('title', 'No Title')} -> {item.get('url', 'No URL')}" for item in results]
    )
    return formatted_results

### Understanding the Final Answer Tool in AI Agents

The `FinalAnswerTool` is a specialized tool used in AI agent frameworks to consolidate the reasoning process and provide a definitive response. Unlike other tools that fetch data or perform computations, this tool acts as a final step where the AI commits to a structured output.

#### Why This Tool Is Important for AI Agents
AI agents often go through multi-step reasoning before arriving at an answer. The `FinalAnswerTool` helps in:

1. Ensuring a structured final response
    - Instead of returning intermediate steps, the agent commits to a well-defined answer.

2. Improving interpretability
    - By using a dedicated final output tool, responses become more consistent and easier to process.

3. Integrating seamlessly into pipelines

    - Many agent frameworks expect a clear final output format—this tool standardizes the output handling.

When designing AI agents, tools like this help bridge the gap between reasoning and action, making interactions smoother and more reliable.

In [7]:
# final answer class tool used after reasoning
class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer

    def __init__(self, *args, **kwargs):
        self.is_initialized = False

# instanciate tool
final_answer = FinalAnswerTool()

### Picking our LLM and initializing it.

To instanciate our agent, we must first create a model instance by setting up a specific `TransformersModel` and its tokenizer (with CUDA enabled and controlled token limits). 

In [ ]:
# pick a model and instanciate it
# model_name= "HuggingFaceTB/SmolLM2-360M-Instruct"
model_name="HuggingFaceTB/SmolLM2-1.7B-Instruct"
# model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
model = TransformersModel(model_id=model_name, device_map="auto", max_new_tokens=200)

### Creating an AI Agent with Custom Tools
Now that we've built our individual tools, the next step is assembling them into an AI agent. This is done using the `CodeAgent` class, which takes a language model (model), a set of tools, and other parameters to define its behavior.

**Key Components of the AI Agent:**

1. `Model` (model) – The LLM that powers the agent’s reasoning and decision-making.
2. `Tools` (tools) – A list of functions the agent can use to extend its capabilities.
3. `Max Steps` (max_steps=6) – Limits how many steps the agent can take before finalizing an answer.
4. `Verbosity` (verbosity_level=1) – Controls the level of detail in logs/debugging output.
5. `Prompt Templates` (prompt_templates) – Provides structured instructions for the agent’s behavior.

In [8]:
# finaly, create the agent and pass the model, tools, number of steps it should reflect for, prompt template...ect

agent = CodeAgent(
    model=model,
    tools=[final_answer,fibonacci_tool, check_NA_market_status, duckduckgo_search], ## add your tools here (don't remove final answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

Finally, we query our `agent` by passing in our queries to the `.run()` method.

In [10]:
print("CodeAgent:", agent.run("Could you give me the 118th number in the Fibonacci sequence?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = fibonacci_tool(n=118)                                                                                   
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 0: Duration 4.45 seconds| Input tokens: 2,511 | Output tokens: 63]

CodeAgent: 2046711111473984623691759


In [11]:
print("CodeAgent:", agent.run("Could you give me the current market price of the stock VEQT.TO?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the current market price of the stock VEQT.TO?                                                │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ticker = "VEQT.TO"                                                                                               
  current_price = check_NA_market_status(ticker=ticker)                                                            
  final_answer(current_price)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Market is open. Current price of VEQT.TO: $46.64

[Step 0: Duration 6.54 seconds| Input tokens: 2,512 | Output tokens: 102]

CodeAgent: Market is open. Current price of VEQT.TO: $46.64


In [9]:
print("CodeAgent:", agent.run("Could you search the web for the latest AI research in 2025"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you search the web for the latest AI research in 2025                                                     │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "Latest AI research 2025"                                                                                
  answer = duckduckgo_search(query=query)                                                                          
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ## Search Results

[The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - 
Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready
-for-today/)
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to 
advanced AI legislation and sustainable AI initiatives.

[AI in the workplace: A report for 2025 | McKinsey - McKinsey & 
Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/superagency-in-the-workplace-empowerin
g-people-to-unlock-ais-full-potential-at-work)
Artificial intelligence has arrived in the workplace and has the potential to be as transformative as the steam 
engine was to the 19th-century Industrial Revolution. 1 "Gen AI: A cognitive industrial revolution," McKinsey, June
7, 2024. With powerful and capable large language models (LLMs) developed by Anthropic, Cohere, Google, Meta, 
Mistral, OpenAI, and others, we have entered a new ...

[Artificial Intelligence | Stanford Emerging Technology 
Review](https://setr.stanford.edu/technology/artificial-intelligence/2025)
• Artificial intelligence (AI) is a foundational technology that is supercharging other scientific fields and, like
electricity and the internet, has the potential to transform societies, economies, and politics worldwide. ... 
Research on foundational AI technologies is difficult to regulate, even among likeminded nations. It is even more 
...

[What's next for AI in 2025 - MIT Technology 
Review](https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/)
AI model makers are also keen to pitch their generative products as research tools for scientists. OpenAI let 
scientists test its latest o1 model and see how it might support them in research. The ...

[Top 15 AI Trends for 2025: Expert Predictions You Need to 
Know](https://techstartups.com/2025/01/01/top-15-ai-trends-for-2025-expert-predictions-you-need-to-know/)
3. Open-Source AI Gains Momentum Last week, we wrote about how Deepseek outperformed OpenAI and Meta's latest 
models at a fraction of the cost.Deepseek, a free open-source AI model developed by a Chinese tech startup, 
exemplifies a growing trend in open-source AI, where accessible tools are pushing the boundaries of performance and
affordability.

[Five Trends in AI and Data Science for 2025 - MIT Sloan Management 
Review](https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2025/)
But we will incorporate the latest research whenever possible. Randy has just completed his annual survey of data, 
analytics, and AI executives, the 2025 AI & Data Leadership Executive Benchmark Survey, conducted by his 
educational firm, Data & AI Leadership Exchange; and Tom has worked on several surveys on generative AI and data, 
technology ...

[Artificial Intelligence News -- 
ScienceDaily](https://www.sciencedaily.com/news/computers_math/artificial_intelligence/)
Jan. 15, 2025 — A new initiative is challenging the conversation around the direction of artificial intelligence 
(AI). It charges that the current trajectory is inherently biased against non ...

[AI Advancements in 2025: The Next Big Innovations to 
Watch](https://techresearchs.com/artificial-intelligence/ai-advancements-in-2025-the-next-big-innovations-to-watch/
)
Artificial Intelligence (AI) continues to evolve, reshaping industries and daily life. In 2025, AI advancements 
will bring groundbreaking innovations, enhancing automation, decision-making, and problem-solving across various 
sectors. This article explores the latest AI advancements in 2025, their applications, and their transformative 
impact on businesses and society.

[10 Artificial Intelligence Trends in 2025 To Stay 
Ahead](https://northwest.education/insights/artificial-intelligence/10-top-artificial-intelligence-trends-to-stay-a
head/)
10 AI trends that will define 2025. The following trends highlight how artificial

[Step 0: Duration 6.77 seconds| Input tokens: 2,510 | Output tokens: 82]

CodeAgent: ## Search Results

[The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready-for-today/)
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to advanced AI legislation and sustainable AI initiatives.

[AI in the workplace: A report for 2025 | McKinsey - McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/superagency-in-the-workplace-empowering-people-to-unlock-ais-full-potential-at-work)
Artificial intelligence has arrived in the workplace and has the potential to be as transformative as the steam engine was to the 19th-century Industrial Revolution. 1 "Gen AI: A cognitive industrial revolution," McKinsey, June 7, 2024. With powerful and capable large language models (LLMs) developed by Anthropic, Cohere, Google, Meta, Mistral, OpenAI, and others, we have entere